Notebook para extraer los datos de Kaggle y transformarlos a tablas para el OLTP

# Fuentes de datos
**Lista de películas y ratings**

- [🎬 MUBI SVOD Platform Database for Movie Lovers](https://www.kaggle.com/datasets/clementmsika/mubi-sqlite-database-for-movie-lovers)

**Género y productora**
- [IMDb Movie and Crew Data](https://www.kaggle.com/datasets/thedevastator/imdb-movie-and-crew-data)

**Reseñas**
- Mubi
- [Rotten Tomatoes movies and critic reviews dataset](https://www.kaggle.com/datasets/andrezaza/clapper-massive-rotten-tomatoes-movies-and-reviews?select=rotten_tomatoes_movie_reviews.csv)

**Usuarios**
- [Social-Media-Users-Dataset](https://www.kaggle.com/datasets/arindamsahoo/social-media-users)

**Vistas**
- Inventada

# Descarga de datos

## MUBI

In [1]:
!kaggle datasets download -d clementmsika/mubi-sqlite-database-for-movie-lovers

Dataset URL: https://www.kaggle.com/datasets/clementmsika/mubi-sqlite-database-for-movie-lovers
License(s): copyright-authors
100% 1.12G/1.12G [00:58<00:00, 28.0MB/s]
100% 1.12G/1.12G [00:58<00:00, 20.5MB/s]


In [2]:
!unzip /content/mubi-sqlite-database-for-movie-lovers.zip -d /content/mubi_data/

Archive:  /content/mubi-sqlite-database-for-movie-lovers.zip
  inflating: /content/mubi_data/mubi_db.sqlite  
  inflating: /content/mubi_data/mubi_lists_data.csv  
  inflating: /content/mubi_data/mubi_lists_user_data.csv  
  inflating: /content/mubi_data/mubi_movie_data.csv  
  inflating: /content/mubi_data/mubi_ratings_data.csv  
  inflating: /content/mubi_data/mubi_ratings_user_data.csv  


## IMDb

In [3]:
!kaggle datasets download -d thedevastator/imdb-movie-and-crew-data

Dataset URL: https://www.kaggle.com/datasets/thedevastator/imdb-movie-and-crew-data
License(s): other
 55% 22.0M/39.7M [00:00<00:00, 98.9MB/s]
100% 39.7M/39.7M [00:00<00:00, 119MB/s] 


In [4]:
!unzip /content/imdb-movie-and-crew-data.zip -d /content/imdb_data/

Archive:  /content/imdb-movie-and-crew-data.zip
  inflating: /content/imdb_data/IMDb movies.csv  
  inflating: /content/imdb_data/IMDb names.csv  


## Rotten Tomatoes

In [3]:
!kaggle datasets download -d andrezaza/clapper-massive-rotten-tomatoes-movies-and-reviews

Dataset URL: https://www.kaggle.com/datasets/andrezaza/clapper-massive-rotten-tomatoes-movies-and-reviews
License(s): CC0-1.0
 98% 149M/152M [00:07<00:00, 23.9MB/s]
100% 152M/152M [00:07<00:00, 20.9MB/s]


In [4]:
!unzip /content/clapper-massive-rotten-tomatoes-movies-and-reviews.zip -d /content/rotten_tomatoes_data/

Archive:  /content/clapper-massive-rotten-tomatoes-movies-and-reviews.zip
  inflating: /content/rotten_tomatoes_data/rotten_tomatoes_movie_reviews.csv  
  inflating: /content/rotten_tomatoes_data/rotten_tomatoes_movies.csv  


## Social Media

In [34]:
!kaggle datasets download -d arindamsahoo/social-media-users

Dataset URL: https://www.kaggle.com/datasets/arindamsahoo/social-media-users
License(s): DbCL-1.0
 69% 2.00M/2.89M [00:00<00:00, 2.92MB/s]
100% 2.89M/2.89M [00:00<00:00, 3.42MB/s]


In [35]:
!unzip /content/social-media-users.zip -d /content/social_media/

Archive:  /content/social-media-users.zip
  inflating: /content/social_media/SocialMediaUsersDataset.csv  


# Importación de librerías

In [40]:
import pandas as pd
import numpy as np
import re

from numpy.random import randint

# Transformación

## Películas_directores y directores

In [10]:
peliculas = pd.read_csv('/content/mubi_data/mubi_movie_data.csv')

In [11]:
peliculas.sample()

,movie_id,movie_title,movie_release_year,movie_url,movie_title_language,movie_popularity,movie_image_url,director_id,director_name,director_url
160030,195796,Sunday Pursuit,1990.0,http://mubi.com/films/sunday-pursuit,en,0,https://images.mubicdn.net/images/film/195796/...,6931,Mai Zetterling,http://mubi.com/cast/mai-zetterling


In [12]:
peliculas.columns

Index(['movie_id', 'movie_title', 'movie_release_year', 'movie_url',
       'movie_title_language', 'movie_popularity', 'movie_image_url',
       'director_id', 'director_name', 'director_url'],
      dtype='object')

Separamos la película, el director y su nombre

In [13]:
pelicula_directores = peliculas[['movie_id', 'director_id', 'director_name']]

Borramos los campos donde hayan nulos

In [14]:
pelicula_directores = pelicula_directores.dropna()

Separamos los ids y nombres de directores, usamos una re para evitar separar nombres que contengan ", Jr.", por ejemplo:
**Arthur Rankin, Jr.**

In [15]:
pelicula_directores['director_id'] = pelicula_directores['director_id'].apply(lambda x: x.split(', '))
pelicula_directores['director_name'] = pelicula_directores['director_name'].apply(lambda x: re.split(r',\s(?!Jr\.)', x))

Eliminamos los que pudieron tener algún error al separar los nombres, pues existen muy pocos casos donde el nombre contiene una coma

In [16]:
pelicula_directores = pelicula_directores[pelicula_directores['director_id'].apply(len) == pelicula_directores['director_name'].apply(len)]

Replicamos cada registro para separar los directores

In [17]:
pelicula_directores = pelicula_directores.explode(['director_id', 'director_name'])

Nos quedamos con la información de los directores y borramos duplicados

In [18]:
directores = pelicula_directores[['director_id', 'director_name']]

In [19]:
directores = directores.drop_duplicates()

Confirmamos que un solo director no tenga diferentes id

In [20]:
directores['director_name'].value_counts()

director_name
Ingo Petzke              9
David Miller             6
Richard Martin           4
Peter Jones              4
Paul Taylor              4
                        ..
Affandi Abdul Rachman    1
Faozan Rizal             1
Steve Gough              1
Ceri Sherlock            1
Ivan Brlić               1
Name: count, Length: 99104, dtype: int64

Nos quedamos con solo un id por director

In [21]:
directores = directores.drop_duplicates(subset=['director_name'])

Cambiamos esos id en las otras tablas

In [22]:
pelicula_directores = pelicula_directores.drop('director_id', axis=1)

In [23]:
pelicula_directores = pelicula_directores.merge(directores, how='left', on='director_name')

Guardamos las tablas

In [24]:
directores = directores.rename({'director_id': 'id_director', 'director_name': 'nombre'}, axis=1)

In [25]:
pelicula_directores = pelicula_directores.drop('director_name', axis=1)

In [26]:
pelicula_directores = pelicula_directores.rename({'movie_id': 'id_pelicula', 'director_id': 'id_director'}, axis=1)

In [27]:
directores.to_csv('./Directores.csv', index=False)

In [28]:
pelicula_directores.to_csv('./Pelicula_director.csv', index=False)

## Películas, Actores, Géneros, País, Productora

In [116]:
peliculas = pd.read_csv('/content/mubi_data/mubi_movie_data.csv')
imdb_peliculas = pd.read_csv('/content/imdb_data/IMDb movies.csv')

Vamos a usar los datos de imdb y rotten_tomatoes para asignar actores, géneros,

In [117]:
peliculas.sample()

,movie_id,movie_title,movie_release_year,movie_url,movie_title_language,movie_popularity,movie_image_url,director_id,director_name,director_url
31888,46185,Maigret and the St. Fiacre Case,1959.0,http://mubi.com/films/maigret-and-the-st-fiacr...,en,2,https://images.mubicdn.net/images/film/46185/c...,32480,Jean Delannoy,http://mubi.com/cast/jean-delannoy


In [118]:
peliculas.columns

Index(['movie_id', 'movie_title', 'movie_release_year', 'movie_url',
       'movie_title_language', 'movie_popularity', 'movie_image_url',
       'director_id', 'director_name', 'director_url'],
      dtype='object')

In [119]:
peliculas_drop = ['movie_url', 'director_id', 'director_name', 'director_url',
                  'movie_title_language', 'movie_popularity', 'movie_image_url']

In [120]:
peliculas = peliculas.drop(peliculas_drop, axis=1)

In [123]:
peliculas.shape[0]

226575

In [124]:
imdb_peliculas.shape[0]

81273

In [125]:
imdb_peliculas.columns

Index(['index', 'imdb_title_id', 'title', 'original_title', 'year',
       'date_published', 'genre', 'duration', 'country', 'language',
       'director', 'writer', 'production_company', 'actors', 'description',
       'avg_vote', 'votes', 'budget', 'usa_gross_income',
       'worlwide_gross_income', 'metascore', 'reviews_from_users',
       'reviews_from_critics'],
      dtype='object')

In [129]:
imdb_keep = ['genre', 'duration', 'country', 'language',
             'writer', 'production_company', 'actors', 'description']

In [130]:
imdb_peliculas = imdb_peliculas[imdb_keep]

In [131]:
df_aux = pd.concat([imdb_peliculas]*3)
df_aux.shape[0]

243819

In [132]:
df_aux = df_aux.reset_index(drop=True)

In [133]:
peliculas = pd.concat([peliculas, df_aux], axis=1)

In [134]:
peliculas = peliculas[~peliculas['movie_id'].isna()]

In [135]:
peliculas.isna().sum()

movie_id                  0
movie_title               0
movie_release_year        5
genre                     0
duration                  0
country                  85
language               2134
writer                 3881
production_company    11670
actors                  166
description            6963
dtype: int64

## Generos, Actores

In [162]:
pelicula_genero = peliculas[['movie_id', 'genre']].astype(str)
pelicula_actor = peliculas[['movie_id', 'actors']].astype(str)

In [163]:
pelicula_genero.sample()

,movie_id,genre
205474,242046.0,Drama


In [164]:
pelicula_actor.sample()

,movie_id,actors
39927,56778.0,"Nikola Djuricko, Sonja Kolacaric, Tara Manic, ..."


Replicamos cada registro para separar los generos y actores

In [165]:
pelicula_actor['actors'] = pelicula_actor['actors'].apply(lambda x: x.split(', '))
pelicula_genero['genre'] = pelicula_genero['genre'].apply(lambda x: x.split(', '))

In [167]:
pelicula_genero = pelicula_genero.explode('genre')
pelicula_actor = pelicula_actor.explode('actors')

Nos quedamos con la información de los generos y acores y borramos duplicados

In [170]:
generos = pelicula_genero[['genre']]
actores = pelicula_actor[['actors']]

In [171]:
generos = generos.drop_duplicates()
actores = actores.drop_duplicates()

In [173]:
generos = generos.reset_index(drop=True)
actores = actores.reset_index(drop=True)

Agregamos un id a cada genero y a cada actor

In [176]:
generos = generos.reset_index()
actores = actores.reset_index()

Sustituimos los nombres por los id generados en las tablas de muchos a muchos

In [178]:
pelicula_genero = pelicula_genero.merge(generos, how='left', on='genre')
pelicula_actor = pelicula_actor.merge(actores, how='left', on='actors')

In [183]:
pelicula_genero = pelicula_genero.drop('genre', axis=1)
pelicula_actor = pelicula_actor.drop('actors', axis=1)

### Guardamos en csv

In [184]:
pelicula_genero.columns

Index(['movie_id', 'index'], dtype='object')

In [185]:
pelicula_actor.columns

Index(['movie_id', 'index'], dtype='object')

In [186]:
pelicula_genero = pelicula_genero.rename({'movie_id': 'id_pelicula', 'index': 'id_genero'}, axis=1)
pelicula_actor = pelicula_actor.rename({'movie_id': 'id_pelicula', 'index': 'id_actor'}, axis=1)

In [188]:
generos.columns

Index(['index', 'genre'], dtype='object')

In [189]:
actores.columns

Index(['index', 'actors'], dtype='object')

In [190]:
generos = generos.rename({'index': 'id_genero', 'genre': 'nombre'}, axis=1)
actores = actores.rename({'index': 'id_actor', 'actors': 'nombre'}, axis=1)

In [193]:
pelicula_genero.to_csv('./Pelicula_genero.csv', index=False)
pelicula_actor.to_csv('./Pelicula_actor.csv', index=False)
generos.to_csv('./Generos.csv', index=False)
actores.to_csv('./Actores.csv', index=False)

## Productora y País

In [205]:
peliculas.columns

Index(['movie_id', 'movie_title', 'movie_release_year', 'genre', 'duration',
       'country', 'language', 'writer', 'production_company', 'actors',
       'description'],
      dtype='object')

In [206]:
peliculas = peliculas.drop(['genre', 'actors', 'language', 'writer'], axis=1)

In [211]:
peliculas['country'] = peliculas['country'].apply(lambda x: str(x).split(', ')[0])

In [212]:
paises = peliculas[['country']]
productoras = peliculas[['production_company']]

In [213]:
paises = paises.drop_duplicates()
productoras = productoras.drop_duplicates()

#Borrar índice anterior
paises = paises.reset_index(drop=True)
productoras = productoras.reset_index(drop=True)

# Asignar índice
paises = paises.reset_index()
productoras = productoras.reset_index()

Cambiamos los nombres en la tabla de peliculas por el índice del país y de la productora

In [ ]:
peliculas = peliculas.merge(paises, how='left', on='country')
peliculas = peliculas.merge(productoras, how='left', on='production_company')

In [218]:
peliculas = peliculas.drop(['country', 'production_company'], axis=1)

In [222]:
peliculas = peliculas.rename({'index_x': 'id_pais', 'index_y': 'id_productora'}, axis=1)

### Guardar resultados

In [227]:
paises = paises.rename({'index': 'id_pais', 'country': 'nombre'}, axis=1)
productoras = productoras.rename({'index': 'id_productora', 'production_company': 'nombre'}, axis=1)

In [229]:
paises.to_csv('./Paises.csv', index=0)
productoras.to_csv('./Productoras.csv', index=0)

## Peliculas

In [230]:
peliculas.columns

Index(['movie_id', 'movie_title', 'movie_release_year', 'duration',
       'description', 'id_pais', 'id_productora'],
      dtype='object')

In [233]:
peliculas = peliculas.rename({'movie_id':'id_pelicula',
                  'movie_title': 'titulo',
                  'movie_release_year': 'anio',
                  'duration': 'duracion',
                  'description': 'descripcion'}, axis=1)

In [234]:
peliculas.to_csv('./Peliculas.csv', index=False)

## Reseñas

In [6]:
resenas = pd.read_csv('/content/mubi_data/mubi_ratings_data.csv')

<ipython-input-6-845f97d6dddc>:1: DtypeWarning: Columns (9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  resenas = pd.read_csv('/content/mubi_data/mubi_ratings_data.csv')


In [13]:
resenas = resenas[~resenas['critic'].isna()]

In [15]:
resenas.columns

Index(['movie_id', 'rating_id', 'rating_url', 'rating_score',
       'rating_timestamp_utc', 'critic', 'critic_likes', 'critic_comments',
       'user_id', 'user_trialist', 'user_subscriber',
       'user_eligible_for_trial', 'user_has_payment_method'],
      dtype='object')

In [19]:
resenas_keep = ['rating_id', 'user_id', 'movie_id', 'rating_score', 'critic', 'rating_timestamp_utc']

In [20]:
resenas = resenas[resenas_keep]

In [24]:
resenas.isna().sum()

rating_id                   0
user_id                     0
movie_id                    0
rating_score            26731
critic                      0
rating_timestamp_utc        0
dtype: int64

In [26]:
resenias = resenas.rename({'rating_id': 'id_resenia',
 'user_id': 'id_usuario',
 'movie_id': 'id_pelicula',
 'rating_score': 'calificacion',
 'critic': 'comentario',
 'rating_timestamp_utc': 'fecha'}, axis=1)

In [32]:
resenias = resenias.drop_duplicates(subset=['id_usuario', 'id_pelicula'], keep='last')

In [33]:
resenias.to_csv('./Resenias.csv', index=False)

## Usuarios

In [97]:
usuarios = pd.read_csv('/content/social_media/SocialMediaUsersDataset.csv')

In [98]:
usuarios = usuarios.drop(['Interests', 'City', 'Country'], axis=1)

In [99]:
def gen_email(x: str):
  x = x.replace(' ', '_') # Turn spaces into _
  x += chr(randint(65, 91)) # Add random upper letter
  x += chr(randint(97, 123)) # Add random lower letter
  x += '_'+str(randint(0,100)) # Add number between 0 and 100
  return x+'@gmail.com'

In [104]:
usuarios['email'] = usuarios['Name'].apply( gen_email )

In [105]:
usuarios.nunique()

UserID    100000
Name       92580
Gender         2
DOB        18536
email     100000
dtype: int64

In [106]:
usuarios.sample(10)

,UserID,Name,Gender,DOB,email
2746,2747,Marcelina Jennings,Female,2001-07-20,Marcelina_JenningsDy_39@gmail.com
91111,91112,Elizabeth Ballintyn,Male,1965-12-25,Elizabeth_BallintynJi_40@gmail.com
53382,53383,Virginia Shurtliff,Male,1977-08-31,Virginia_ShurtliffHl_92@gmail.com
94778,94779,Adam Jones,Female,1977-07-31,Adam_JonesXt_65@gmail.com
90683,90684,Jose Williams,Male,1997-06-25,Jose_WilliamsVs_26@gmail.com
64733,64734,Brandon Hall,Male,1968-07-29,Brandon_HallGe_46@gmail.com
9535,9536,Alfredo Riddle,Male,2003-08-07,Alfredo_RiddleTo_35@gmail.com
19526,19527,Roy Fox,Female,1961-03-25,Roy_FoxEw_89@gmail.com
54735,54736,Shannon Weaver,Female,1979-04-03,Shannon_WeaverBj_95@gmail.com
56792,56793,Doris Michelson,Male,1958-10-10,Doris_MichelsonNt_69@gmail.com


Solo falta hacer que los id coincidan con los id en ratings

In [107]:
resenias.nunique()

id_resenia      748865
id_usuario       37522
id_pelicula      69219
calificacion         5
comentario      657467
fecha           717309
dtype: int64

In [108]:
usuarios_con_resenia = resenias[['id_usuario']]

In [109]:
usuarios_con_resenia = usuarios_con_resenia.drop_duplicates()

In [112]:
usuarios_con_resenia = usuarios_con_resenia.reset_index(drop=True)
usuarios_con_resenia = usuarios_con_resenia.reset_index()

In [116]:
usuarios = usuarios.merge(usuarios_con_resenia, how='left', left_on='UserID', right_on='index' )

In [118]:
usuarios['id_usuario'] = usuarios['id_usuario'].fillna(usuarios['UserID'])

In [121]:
usuarios = usuarios.drop_duplicates(subset=['id_usuario'])

In [123]:
usuarios = usuarios.drop(['UserID', 'index'], axis=1)

In [125]:
usuarios.columns

Index(['Name', 'Gender', 'DOB', 'email', 'id_usuario'], dtype='object')

In [127]:
usuarios = usuarios.rename({'Name': 'nombre_usuario',
                  'Gender': 'genero',
                  'DOB': 'fecha_nacimiento',},
                axis=1
                  )

In [128]:
usuarios.to_csv('./Usuarios.csv', index=False)